In [ ]:
#Função de avaliação
def avaliar_modelo(modelo,x_train,y_train, x_test, y_test):
    """
    Calcula e exibe as métricas de avaliação (R², MAE, RMSE) para um modelo de regressão.
    """
    # Fazer previsões com os dados de teste
    previsao_treino = modelo.predict(x_train)
    previsao_teste =  modelo.predict(x_test)
    
    # Calcular as métricas de treino
    mae_train = mean_absolute_error(y_train, previsao_treino)
    rmse_train = np.sqrt(mean_squared_error(y_train, previsao_treino)) # RMSE é a raiz quadrada do MSE
    r2_train = r2_score(y_train, previsao_treino)
    
    # Calcular as métricas de teste
    mae_test = mean_absolute_error(y_test, previsao_teste)
    rmse_test = np.sqrt(mean_squared_error(y_test, previsao_teste)) # RMSE é a raiz quadrada do MSE
    r2_test = r2_score(y_test, previsao_teste)
    
    # Concatenar resultados
    resultados = {
        'Métrica': ['R²', 'MAE', 'RMSE'],
        'Treino': [r2_train, mae_train, rmse_train],
        'Teste': [r2_test, mae_test, rmse_test]
    }
    resultado = pd.DataFrame(resultados)
    return round(resultado,3)

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna
import joblib

In [ ]:
df = pd.read_csv('../data/processed/df_ML.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df.sort_index(inplace=True)

In [ ]:
# Dividindo cronologicamente
df_train = df[df.index.year < 2017]
df_valid = df[df.index.year == 2017]

# Separando features
x_train = df_train.drop('sales', axis=1)
y_train = df_train['sales']
x_valid = df_valid.drop('sales', axis=1)
y_valid = df_valid['sales']

In [ ]:
def objective(trial,x_treino,y_treino,x_validacao,y_validacao):
    params = {
        'objective': 'regression_l1',
        'metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 800, 2500),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'random_state': 42,
    }
    
    # Treinar o modelo com os parâmetros sugeridos
    model = lgb.LGBMRegressor(**params)
    model.fit(
        x_treino, 
        y_treino,
        eval_set=[(x_validacao, y_validacao)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(100),optuna.integration.LightGBMPruningCallback(trial, "rmse")]
    )
    
    # Fazer previsões e avaliar
    previsoes = model.predict(x_validacao)
    rmse = np.sqrt(mean_squared_error(y_validacao, previsoes))
    
    return rmse

In [ ]:
#Definição do criterio de estudo
study = optuna.create_study(direction="minimize")

# Iniciar a otimização
study.optimize(
    lambda trial: objective(trial, x_train, y_train, x_valid, y_valid),
    n_trials=50
)

best_params = study.best_params

In [ ]:
# Melhores resultados
print(best_params)
print(f"\nMelhor valor de RMSE: {study.best_value}")

In [ ]:
# Treinando modelo otimizado
lgbm_otimizado = lgb.LGBMRegressor(**best_params, random_state=42)
lgbm_otimizado.fit(x_train, y_train)

In [ ]:
path_lgbm_otim = '../models/lgbm_otimizado.joblib'
lgbm_otimizado = joblib.load(path_lgbm_otim)

In [ ]:
# Avaliando performance
avaliar_modelo(lgbm_otimizado, x_train, y_train, x_valid, y_valid)

In [ ]:
# Salva o  melhor modelo
import joblib
joblib.dump(lgbm_otimizado, 'lgbm_otimizado.joblib')

In [ ]:
# Salva featrures do modelo
features_do_modelo = x_train.columns.tolist()
joblib.dump(features_do_modelo, 'lista_de_features.joblib')